## Fetch Open Street Map (OSM) data

► *This procedure is based on the University of Helskinki tutorial ([LINK](https://automating-gis-processes.github.io/site/notebooks/L6/retrieve_osm_data.html)) and the OSMNX documentation site ([LINK](https://geoffboeing.com/2016/11/osmnx-python-street-networks/))*

OpenStreetMap (OSM) is a global collaborative (crowd-sourced) dataset and project that aims at creating a free editable map of the world containing a lot of information about our environment. It contains data for example about streets, buildings, different services, and landuse to mention a few. You can view the map at www.openstreetmap.org. You can also sign up as a contributor if you want to edit the map.

OSM has a large userbase with more than 4 million users and over a million contributers that update actively the OSM database with 3 million changesets per day. In total OSM contains 5 billion nodes that form the basis of the digitally mapped world that OSM provides (stats from November 2019).

OpenStreetMap is used not only for integrating the OSM maps as background maps to visualizations or online maps, but also for many other purposes such as routing, geocoding, education, and research. OSM is also widely used for humanitarian response e.g. in crisis areas (e.g. after natural disasters) and for fostering economic development (see more from Humanitarian OpenStreetMap Team (HOTOSM) website).

The `osmnx` package is an invaluable tool for extracting data from OSM. ([Read more...](https://geoffboeing.com/2016/11/osmnx-python-street-networks/). It works by defining an area, which can be done easily with its build in geocoder, and using that area to extract features into a [GeoPandas](https://geopandas.org/) [geodataframe](https://geopandas.org/data_structures.html) or as a [NetworkX](https://networkx.github.io/documentation/stable/) [graph](https://networkx.github.io/documentation/stable/reference/introduction.html#graphs) object (both of which we will discuss later). 

### Importing packages
* It all begins with importing the osmnx and matplotib plotting libraries

In [ ]:
#Import the osmnx library
import osmnx as ox

#Import matplotlib's plotting library
import matplotlib.pyplot as plt

#Configure Jupyter to show plots in the document itself
%matplotlib inline

### Set the analysis extent
* We begin by establishing our area of interest. This is a Shapely geometry object that defines the spatial extent of the OSM data we extract. The osmnx's `gdf_from_place()` geocodes a place name, storing the results in a geodataframe. Ultimately, we'll want to do all of North Carolina, but to speed things up, we'll just do a single county. 

In [ ]:
#Create a geodataframe of Durham County
durm_area = ox.gdf_from_place("Durham County, NC")
type(durm_area)

In [ ]:
#Examine the resulting geodataframe
durm_area.head()

In [ ]:
#Plot the resulting dataframe
durm_area.plot();

* The osmnx functions we'll use require a Shapely geometry object, not a geodataframe, so we'll extract the one and only shape - stored in the geodataframe's `geometry` column - into a new variable. 

> We'll talk about geodataframes and shapely geometry objects in more detail next week

In [ ]:
#Extract the shapely polygon from the returned DF
durm_poly = durm_area.at[0,'geometry']

In [ ]:
#Display the shape
durm_poly

### Fetch amenities
Next, we'll fetch a set of amenities falling within our Shapely polygon. The steps here include:
* Examine the list of amenities: https://wiki.openstreetmap.org/wiki/Key:amenity
* Construct a Python list of the amenities you want to extract
* Search for these amenities using the `ox.create_poi_gdf()` function, returning the results as a geodataframe...

In [ ]:
#Create a Python list of all the amenities we want to find 
amenities_list = ['cafe','restaurant']

In [ ]:
#Use the `pois.create_poi_gdf() function to extract amenities within our polygon
durm_amenities = ox.pois.create_poi_gdf(polygon=durm_poly,amenities=amenities_list)
type(durm_amenities)

In [ ]:
#Reveal how many features were extracted
len(durm_amenities)

In [ ]:
#Have a peek
durm_amenities.head()

In [ ]:
#Save to csv file
durm_amenities.to_csv("Durham_Amenities.csv",index=False)

In [ ]:
#Plot the sites on a map
fig, ax = plt.subplots(figsize=(12,8))
durm_amenities.to_crs(epsg=3857).plot(ax=ax)
import contextily as ctx
ctx.add_basemap(ax)

### Fetch roads - as a graph
The `osmnx` package has another function - [`graph_from_polygon`](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.core.graph_from_place) - that downloads features as a graph. 
Next, we examine how we can fetch graph data from OSM. 
https://github.com/gboeing/osmnx-examples/blob/master/notebooks/00-osmnx-features-demo.ipynb

In [ ]:
#Fetch highways as 
highways = ox.graph_from_polygon(durm_poly,  
                                 network_type='drive',
                                 simplify=True,
                                 retain_all=False,
                                 truncate_by_edge=True,
                                 infrastructure='way["highway"~"motorway"]')

In [ ]:
#Check the data type of the new object
type(highways)

In [ ]:
#Create a folder to hold the OSM data (if it doesn't exist already)
import os
if not os.path.exists('osm_data'): os.mkdir('osm_data')

In [ ]:
#Save the graph as a "graphml" file
ox.save_graphml(highways,filename="Durham_road_net.graphml",folder='osm_data')

In [ ]:
#Convert to shapefile
ox.save_load.save_graph_shapefile(highways,filename='DurhamRoads.shp',folder='osm_data')

In [ ]:
#Zip the folder
import shutil
shutil.make_archive('DurhamRoads','zip','osm_data')

Read in the graph, project it and display it. 

In [ ]:
#Read the graph file back in as a graph
durm_graph = ox.save_load.load_graphml('Durham_road_net.graphml',folder='osm_data')

In [ ]:
fig, ax = ox.plot_graph(durm_graph)